In [1]:
import boto3
import time
import os
import re
from decimal import Decimal, InvalidOperation

from dotenv import load_dotenv

from SeekerTruther import Seeker, Truther
from KnowledgeGraph import KnowledgeGraph
from SeekerTruther import Seeker, Truther
from RewardFunction import RewardFunction

In [2]:
load_dotenv()
textract = boto3.client(
    'textract',
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    region_name='ap-southeast-2'
)

In [3]:
#bucket_name = 'knowledge-graph-test-examples'
#document_name = 'KPMG-TL-FDD-Report-Aug-2020.pdf'
#
#response = textract.start_document_analysis(
#    DocumentLocation={'S3Object': {'Bucket': bucket_name, 'Name': document_name}},
#    FeatureTypes=['TABLES', 'LAYOUT']
#)
#
#job_id = response['JobId']
#print(f"Started Textract job with JobId: {job_id}")
#
#while True:
#    status = textract.get_document_analysis(JobId=job_id)
#    status_value = status['JobStatus']
#    if status_value in ['SUCCEEDED', 'FAILED']:
#        print(f"Job status: {status_value}")
#        break
#    print("Waiting for job to complete...")
#    time.sleep(5)
#
#all_blocks = []
#if status_value == 'SUCCEEDED':
#    next_token = None
#    while True:
#        response = textract.get_document_analysis(JobId=job_id, NextToken=next_token) if next_token else textract.get_document_analysis(JobId=job_id)
#        all_blocks.extend(response['Blocks'])
#        if 'NextToken' not in response:
#            break
#        next_token = response['NextToken']

In [4]:
job_ids = [
    "1b0741a40f6d4da52a7aa9bec159b7d65ca6fdc9558d2e87e4662c5b9393c4ad",
    "ec4d2cb668d6615cb2d3b82c8d2158e37593ce3a74b84e50ea11f2bc028f20c4",
    "4f9766964c48e72adef6c9fbed4e9b360d4281528e9914887fa4800e641ba400",
    "a779934c96f40cc210dbb78d685c50f320294bd3af78c6dcd7b9fb491e8658bf",
    "b4920720da8a49c63cb13286364165957a2b4be703d3ded76718bd028d3245ae",
    "88cb53d622cdc4c1af238d68e1aaee4b87989089d3bbf96afc5a16f380e646ef",
    "83107006e337fd77085e58aaf27462a6164fc22ee9879fab4b0b623826ceaa5e",
    "6a061892833c4ef0f7a3c939ea45097d92a90d40004b096112ad7cd477eeb69a",
    "446770d51f5214819fe06954c6d3676cc791eb7a7fdd43d91d0379d02397941d",
    "ebb39c66f25112579c0462754d14decaf87c92729447ccc7fb9701d94bebecb3",
    "0acfaefe45bd51ba0427cf7ec946d634c03f2b525157f944ac74175df2ddf5be",
    "f1b841b7f9fc4317250b34e23d9941068ec713def775bb9a7d3f85fe124580e2",
    "b4558a7cc0b2472a3cf8333477e6ce2c1f40d6916a8e27b3f37398d568efd872",
    "2937f4582944ebeacfde8b321f442826d64dd1e64910b17c3ce4c4118b596e89",
    "0c4096184cb89cb2eef5387215366689e702b67b3f0ebeeec955eece176bc2d5",
    "49fa7abdc6cb8aeef13e173c90bf751cd4ceee105219011333bb0ed5b26dfc4f",
    "08a79e8f3aedacd61424533622e4d44a6390162584be999e5b28c7a4de554fdf",
    "8a09eebbc8e0eba04b2787bad8c01d0554f7cc68d80cd6d8018247d1965ef526",
    "5e8736a94a0df972bd834ee573383d0dce9a65ab1b50b8dfd8be6c1d8ecdeb74",
    "038d9b3ea53aa0dbb97b0c37c85203dc4c770ece5735e5a201cd1b902573b2cf",
    "57c949d26e9c095e8a7efc85f0fe4de1a30242a68aedc99c7766ddc19c2141c8",
    "5cc135fac7c02e2de9e1a600c1a14b1ab760d0b37397fb9d7c19a2d4979d6034",
    "6a246c494a9bbe38632cf3dcf441846e59ebb419d4d10cc5eed598d54ebcb795",
    "cd5c6637adb27f6b06a1bc93300030400c408edb940b6979d4494625d9b41e46",
    "326667e631463f25b33738709e47d420c11695e536a1590ddde5d23d0bf572af",
    "b92bc218de838328748bb83797c9dd1ec0039dc0f3444ff99112d9fd434a7268",
    "2a549384025ea44a20335b4c2305d1d8bb543b90fd12c3e34f5484650b85f048",
    "6616bc72aba76c32196e787e8435e15ee88e284f211d0116c91fe9b4ce8f96f1",
    "37be9f5aba7c141a85515e231b1452a3e27047b981a4939a9c28d5d7a9e0c2ef",
    "3574aec71a38c942c41c41c2dc9da624b915b3f5910f19b09eecfa17d797f469",
    "517e1e8ab07c24a9a20999f89fa4aa53cc0b7295fee1c7634b88a41586f7df4a",
    "2a97ff09c1aeca191b41323f8ee090c32cbf36c29a5a5455b01aaa8dc8e384e8",
    "07875c7894e1884b46c42f010b837e8b36e33081cf190e9976e893d14338f925",
    "1dd66ea871eb54384b17e8edb8ca512c1b0723d93d17cc8af7e4eb9d3cdc99aa",
    "71781aaf83a53a06f08bc7bf40aa4cc82d4e786236227829ca224ffd2b5aae43",
    "57a123753ea99b9ee3ad7c022d648a88e1aadc82d09481af33381b27c59a6521",
    "d385e9e4499d2439a863ee998ee98d388eb92001ea4e3fd1480da109a47f5226",
    "fb56642e0344b561c5d1825abd484e28a16fd27ee06e27a4e642b396a22fc324",
    "68c29367b8762b917e5bce6e9d87d83f043a67232376398b022ef7135f1218c4",
    "b15d95be04c92ecd6ba1fed9146f29cbe8e566e849c97c68a9b322c81a716a3f",
    "08de52db25ef0968acbfd0915ec42dbf6d242a04c154ed8f9656833eb980293f",
    "b12d9f7c17737ee4e00c6cbadf4b22978e071507af83c4d582cb055ca52c8793",
    "1cdad778ff301a32595ea415f9011738cf19ad54160c90f96cfda49d2a3474db",
    "3484c08ba821fc9208f985194dcf2074cf5dc6439af871aeb8e38be360feadc6"
]

In [5]:
all_blocks = []
for idx, job_id in enumerate(job_ids):
    print("Attempting to retrieve job_id", job_id)
    next_token = None
    sec_pag = False
    while True:
        if sec_pag: print("We are doing a second pag")
        response = textract.get_document_analysis(JobId=job_id, NextToken=next_token) if next_token else textract.get_document_analysis(JobId=job_id)
        for b in response['Blocks']:
            b['Page'] = idx + 1
        all_blocks.extend(response['Blocks'])
        if 'NextToken' not in response: break
        next_token = response['NextToken']
        sec_pag = True

Attempting to retrieve job_id 1b0741a40f6d4da52a7aa9bec159b7d65ca6fdc9558d2e87e4662c5b9393c4ad
Attempting to retrieve job_id ec4d2cb668d6615cb2d3b82c8d2158e37593ce3a74b84e50ea11f2bc028f20c4
Attempting to retrieve job_id 4f9766964c48e72adef6c9fbed4e9b360d4281528e9914887fa4800e641ba400
Attempting to retrieve job_id a779934c96f40cc210dbb78d685c50f320294bd3af78c6dcd7b9fb491e8658bf
Attempting to retrieve job_id b4920720da8a49c63cb13286364165957a2b4be703d3ded76718bd028d3245ae
Attempting to retrieve job_id 88cb53d622cdc4c1af238d68e1aaee4b87989089d3bbf96afc5a16f380e646ef
Attempting to retrieve job_id 83107006e337fd77085e58aaf27462a6164fc22ee9879fab4b0b623826ceaa5e
Attempting to retrieve job_id 6a061892833c4ef0f7a3c939ea45097d92a90d40004b096112ad7cd477eeb69a
Attempting to retrieve job_id 446770d51f5214819fe06954c6d3676cc791eb7a7fdd43d91d0379d02397941d
Attempting to retrieve job_id ebb39c66f25112579c0462754d14decaf87c92729447ccc7fb9701d94bebecb3
Attempting to retrieve job_id 0acfaefe45bd51ba0427

In [6]:
def is_number_like(value: str) -> bool:
    """Returns True if the value is 'number-like'."""
    
    # Strip whitespace
    value = value.strip()

    # Check for empty string
    if not value:
        return False
    
    # Handle percentages: remove percentage sign and check if the remaining string is a number
    if value.endswith('%'):
        value = value[:-1]  # Remove the '%' sign
        try:
            Decimal(value)  # Try parsing as a decimal
            return True
        except InvalidOperation:
            return False

    # Handle currency symbols: remove common symbols like £, $, €, etc.
    value = value.replace('£', '').replace('$', '').replace('€', '').replace('₹', '')
    
    # Remove commas for numbers with thousands separators
    value = value.replace(',', '')
    
    # Handle 'million', 'billion', etc. (by multiplying by corresponding factors)
    multipliers = {
        'million': 1_000_000,
        'billion': 1_000_000_000,
        'thousand': 1_000,
        'k': 1_000,
        'm': 1_000_000,
        'b': 1_000_000_000,
    }
    
    # Check if the string ends with one of the multipliers
    for suffix, multiplier in multipliers.items():
        if value.lower().endswith(suffix):
            try:
                num_part = value.lower().replace(suffix, '').strip()
                num = Decimal(num_part) * multiplier
                return True
            except InvalidOperation:
                return False

    # Handle cases like "4.6x" (multiplication or "times" notation)
    if value.endswith('x') or value.endswith('X'):
        value = value[:-1]  # Remove the 'x'
        try:
            Decimal(value)  # Try parsing as a decimal number
            return True
        except InvalidOperation:
            return False

    # Attempt to parse the value as a decimal or integer
    try:
        Decimal(value)  # Try parsing as a decimal number
        return True
    except InvalidOperation:
        return False
    
def normalize_number(value: str) -> Decimal:
    value = value.strip()
    if not value:
        return Decimal('NaN')
    if value.endswith('%'):
        value = value[:-1]
        try:
            return Decimal(value) / 100
        except InvalidOperation:
            return Decimal('NaN')
    value = value.lstrip('+')
    value = value.replace('£', '').replace('$', '').replace('€', '').replace('₹', '')
    value = value.replace(',', '')
    multipliers = {
        'million': 1_000_000,
        'billion': 1_000_000_000,
        'thousand': 1_000,
        'k': 1_000,
        'm': 1_000_000,
        'b': 1_000_000_000,
    }
    for suffix, multiplier in multipliers.items():
        if value.lower().endswith(suffix):
            num_part = value.lower().replace(suffix, '').strip()
            try:
                return Decimal(num_part) * multiplier
            except InvalidOperation:
                return Decimal('NaN')
    if value.endswith('x') or value.endswith('X'):
        value = value[:-1]
        try:
            return Decimal(value)
        except InvalidOperation:
            return Decimal('NaN')
    try:
        return Decimal(value)
    except InvalidOperation:
        return Decimal('NaN')

In [7]:
labels = {
    "b5563997-da1f-4d1a-84d1-59b7d9892c91":["2e85428c-0e6c-41e6-987b-1e5cf9dd969d","8423e0b4-9838-4102-bb73-708b012882c9"],"7d1442aa-e728-4f77-b1c8-5c81075101ec":["94c9987c-2741-45b7-8021-7350bc7ffdf7","5f26d41b-49fc-40b9-bbd9-672a08af201d"],"37d39883-b0c1-4b72-8e62-f4fb5a67f2a3":["d0781a16-7469-426c-b461-cc4f3504e0f8","a7763578-d600-4199-a302-41dc85cc17dd"],"3eea16e7-4932-4865-9112-ffe13e154d2b":["bf2e6a08-4d43-4bdf-88ca-60b2a37dbc6b","4d31f575-c8d4-4d8e-a809-e3ae73c955c3","1f02ab13-b8db-43c6-891d-7495d2443bb2","ff50694f-afda-4369-ae71-30cd0ee0f7b4"],"b4fbe961-7805-4517-b870-e6a19a788c1c":["408530d2-7d89-4df0-a6fa-1daa1830e0d0","44c46f06-e3c5-449f-80ef-92c26bbfac4a","370c3034-2bce-4a48-84d8-7f541235b12b","8a81738b-c37c-4caa-855f-1c5fc0087478"],"1b1126f8-7ac5-472d-90ae-79e300216e60":["a2e97ce5-a3ae-44b9-99cb-a2a1dea0251b","b63f9773-09e7-4c85-ad76-f8004ade2ea0","a9a618eb-6b13-405e-9702-b416fa2b746e"],"c8e98593-47c6-43cc-ad17-ed2aca077c5a":["9cc10411-630a-476c-9aa9-5dd30929a915","98be2ab7-6a20-4af3-bd74-0a3cfc85298b","0232d491-0bab-49f1-889f-863590da6d11"],"19b4e4a1-bceb-4831-b92a-d6cfa78f9582":["d0781a16-7469-426c-b461-cc4f3504e0f8","a7763578-d600-4199-a302-41dc85cc17dd"],"893dda63-6897-4e5d-abe4-e267e78615a5":["bf2e6a08-4d43-4bdf-88ca-60b2a37dbc6b","4d31f575-c8d4-4d8e-a809-e3ae73c955c3","1f02ab13-b8db-43c6-891d-7495d2443bb2","ff50694f-afda-4369-ae71-30cd0ee0f7b4"],"c5b9fdaf-15d5-4f27-998e-7737548031aa":["38cdf47b-722c-4987-b6d7-c9e4d42f25b3","556a19b0-aef5-4e41-8e65-7d630dc62923","4aae841d-8b08-4f85-9deb-acbbf1867b18","8def071c-f2ac-4a66-bdf2-e575c4212af2","73d2685f-f5b1-4036-8c4b-69e3f5a35770"],"b01d7f23-be01-45c9-a928-fd7f3ce93d69":["be2a868c-a710-4103-bb53-4ad52bbfcfa4","8fc1271f-083c-4431-9aa7-fab2adececf9","830def7e-8542-4354-ba62-dbee30fe89d8","37454792-5df9-4f89-9310-460d18c722d1"],"df53e93f-4829-4bdb-a01c-a61f1c09acca":["50bbc11c-4c06-4cff-8a75-68bb3e5b99bd","530eb85f-fda0-459a-93ad-9add7c861f3d","53967f07-4de8-4e5b-8fea-18e267a1f431","92fb9ea4-710c-4e23-93b5-2f002afad3b2","529fa439-42a0-493e-9327-ef4a906b0b1d"],"81eea891-f0df-430f-a90a-9896c7623c0e":["38cdf47b-722c-4987-b6d7-c9e4d42f25b3","556a19b0-aef5-4e41-8e65-7d630dc62923","73d2685f-f5b1-4036-8c4b-69e3f5a35770","4aae841d-8b08-4f85-9deb-acbbf1867b18","8def071c-f2ac-4a66-bdf2-e575c4212af2"],"4360ddc9-8613-4228-a970-2f549c4da769":["38cdf47b-722c-4987-b6d7-c9e4d42f25b3","73d2685f-f5b1-4036-8c4b-69e3f5a35770","556a19b0-aef5-4e41-8e65-7d630dc62923","4aae841d-8b08-4f85-9deb-acbbf1867b18","8def071c-f2ac-4a66-bdf2-e575c4212af2"],"7323a7ec-2e7a-4fec-b47c-a8e9ce5a56e5":["5ac4be86-9f32-4282-b35b-eea3b2e5f1f3"],"97b8f837-fb09-48dc-ae54-1e5166f61f9a":["6c94d97d-9f21-4ceb-8930-8ceb72df7d5b"],"27190de9-8089-4a0e-824d-cd48f0fff034":["f72b0e9f-a82a-4fa0-a885-ee6a55c53d54"],"57d15276-be0c-45ba-9afb-8dfce91e9733":["fe9ad850-9f84-4661-a766-3d3db2422e23"],"83e9da13-3a4c-4cf0-baf8-b75145bf99c1":["4e7f36f3-8aa5-49d5-b14f-6f8b17fc672e"],"4607bd97-d436-4e59-9feb-77d527c08d09":["6c0a5cfb-0340-4679-bf5c-ffd3b05379c1"],"6c3a0a0d-709d-4e65-858e-c4b95946f7d3":["6c0a5cfb-0340-4679-bf5c-ffd3b05379c1"],"c71c762a-ee0f-4431-b72c-9d975b5ac1e0":["2b47590b-34b7-4a93-8daa-814c437a6346"],"31411fb2-1eaf-4537-9c96-84a4e1569bb2":["93e62f7d-f1a8-42b7-a232-5832c6eb27d5","0ce5d951-ccf5-4c00-91a1-221f4dbf8e57"],"2dfe76ef-c88a-40b2-9b63-170db54b0cd0":["50278fd6-d6ea-4d8f-8680-9783386c487e"],"139951f0-7189-425a-9473-5a386a25aeb4":["8d160789-1ee3-4c05-a01b-a1a8775fb262","8fae892b-419e-4da0-898d-612f173d14e5"],"2796b338-cef9-425f-97c4-86b9a488a8e6":["5b7e369d-3e95-4d5d-ade4-c9b3c394dd8b","fa7e9c91-d92b-41b9-8725-9a471c3ba960"],"f28651f2-d68b-44f2-9033-f8342118c235":["3ef7ff09-e4b4-4383-8720-86f352817546","725082f5-6b86-43c5-9820-1de554efc409"],"bc389554-5e4d-4b5c-99fe-c85751a865df":["eaf34fa8-ddc4-40b1-9235-9b382718842d"],"bbc90001-fc51-4f78-af2b-24e523502c82":["5402f4a4-0741-4593-84a2-da614eae8126"],"d555082a-bbd1-4a09-8cf9-b1d0fb800775":["5402f4a4-0741-4593-84a2-da614eae8126"],"69fb2de9-a019-4c96-8fb2-5297c4e75f91":["eaf34fa8-ddc4-40b1-9235-9b382718842d"],"482256ef-1a50-4dfa-bbb5-d4f7393d4b24":["2e85428c-0e6c-41e6-987b-1e5cf9dd969d","8423e0b4-9838-4102-bb73-708b012882c9"],"cd99e439-9d01-4dd3-8342-dc08bf8d7c1e":["94c9987c-2741-45b7-8021-7350bc7ffdf7","5f26d41b-49fc-40b9-bbd9-672a08af201d"],"502eae41-76e0-42a8-85db-ebbf7add2077":["d0781a16-7469-426c-b461-cc4f3504e0f8","a7763578-d600-4199-a302-41dc85cc17dd"],"59d5bd8e-1678-42fb-9f99-c8412d2d47d7":["bf2e6a08-4d43-4bdf-88ca-60b2a37dbc6b","4d31f575-c8d4-4d8e-a809-e3ae73c955c3","1f02ab13-b8db-43c6-891d-7495d2443bb2","ff50694f-afda-4369-ae71-30cd0ee0f7b4"],"c0f77425-9878-4945-a681-100bfc6fe6c9":["9ade8893-9389-43ec-84bb-feea687d6b48","7ac3aa71-ffec-47fb-9153-43f0a97c4a04","287e3f93-5c96-464a-8326-7b66381c861d"],"e9999772-942b-4808-beae-fa52c44e7a51":["408530d2-7d89-4df0-a6fa-1daa1830e0d0","370c3034-2bce-4a48-84d8-7f541235b12b"],"7e4759ec-8c5f-458e-9e81-bcc0b46405aa":["1484d051-c2fb-4804-af01-f88b50b653db","8fbc7ed9-d16f-4f6e-b870-b3af507792c3"],"c9d58cb4-984a-4c2d-931f-ed676a35f72e":["2e85428c-0e6c-41e6-987b-1e5cf9dd969d","8423e0b4-9838-4102-bb73-708b012882c9"],"99e646e8-d32f-4676-b959-ce334b149de3":["94c9987c-2741-45b7-8021-7350bc7ffdf7","5f26d41b-49fc-40b9-bbd9-672a08af201d"],"e7ce5cca-5cc1-4855-9248-ecb254f3177a":["ab4230b3-f8f6-4775-a66c-a4510780c74b","382b699d-2835-4e59-9890-8431e4422f11"],"2ae03cc0-77c6-4afc-9cfa-f4887a556a64":["c93c9b0e-8ff0-4a43-a72f-a312c7b62789","31a89cd1-5c75-4b09-9869-96af4d7d130a","0891b295-968e-4624-9e2e-f319a963282c"],"09d36d16-3a29-45ed-88e6-b3b12c51d76b":["d0781a16-7469-426c-b461-cc4f3504e0f8","a7763578-d600-4199-a302-41dc85cc17dd"],"fb6ffef6-6176-4186-9760-371202be2c5d":["bf2e6a08-4d43-4bdf-88ca-60b2a37dbc6b","4d31f575-c8d4-4d8e-a809-e3ae73c955c3","1f02ab13-b8db-43c6-891d-7495d2443bb2","ff50694f-afda-4369-ae71-30cd0ee0f7b4"],"13fce038-6145-4726-9c0a-d2cea1a9179e":["5ac4be86-9f32-4282-b35b-eea3b2e5f1f3"],"6edb864e-dda6-42dd-8328-15e140d681d3":["bb71d781-8ff4-4880-beb2-eb5e305d9b12"],"0fd785a2-0554-4886-9d2d-ad9ac259ccc3":["4a7c25ef-ba36-4152-8fa8-7d816046deaa"],"27f6259f-1092-4b4b-98c2-a0021c15ac75":["65b64f86-da13-4be0-9e7e-ada73a8bf89c","6c94d97d-9f21-4ceb-8930-8ceb72df7d5b"],"6aaf030a-888f-4702-b6c8-a6a5337c00f3":["38a29c8b-5d61-4c7e-be39-d61e044a969a"],"c94718fa-7d78-4255-b808-11296c8a55ef":["7cd85ab1-7897-44a4-9c22-732394d0c594"],"913888f2-87dd-4ca5-935e-394c4b4a547a":["58642867-a2e8-4da9-8c0d-b7fac2295147"],"aa52529c-1786-4d71-9f07-2eb9c05949dc":["7d13013f-ec27-4d50-9624-7625f9863356"],"1f2da07f-ceaf-4396-af0f-f1b31b151ce2":["eab98a5e-8e06-42e2-b0ee-5a1335faf072"],"7c366375-6f44-4685-b660-71a5a6f13b22":["dc87f7be-8510-4bb4-afc2-9ed68a6c42cb"],"90f1d9ee-c4b5-4324-93e0-74ceb67774b3":["f72b0e9f-a82a-4fa0-a885-ee6a55c53d54"],"798693c2-079b-4deb-908a-5dbab1d267c8":["dd60b2c7-e937-4951-86ff-7b00345f3d0f"],"47c0a212-c1d8-4c5e-888b-265cbe8ec3f4":["a325d7df-6e00-459d-8a98-6ebfe651634c"],"5aee1576-1361-491c-80e4-fd80569b92f4":["fe9ad850-9f84-4661-a766-3d3db2422e23"],"6d062ca1-07f8-41a1-8cc1-58e5293a885d":["bd6cff24-ff39-4514-b20d-3c335a4dd33b"],"446b0646-2ed6-441e-a38c-8416beb2ef1a":["933541a6-1fba-4fde-816d-8aeeb18cd41f"],"d9963ab6-0322-49c5-bc07-cadcfccfcf39":["d3eb8b1f-3130-46e7-bc99-0eb13b4a97b5"],"edcb9b7b-2c67-47d1-80fa-1637e93dbf28":["dc87f7be-8510-4bb4-afc2-9ed68a6c42cb"],"59ae0876-bddd-4063-8705-d72bbda7ee4c":["eab98a5e-8e06-42e2-b0ee-5a1335faf072"],"cc9d4440-8033-493d-a70e-40b3d98a4585":["5b3969ab-321c-433c-9a55-b5016216c31c"],"82708a47-87f7-4285-b664-4faa913c4386":["6bb9d7d4-e8a4-4e35-9ca1-077e394d843c","e72b1d77-c4bc-42e8-b30e-0c166f1cae04"],"b7b4101f-91e8-461a-8ee5-d245544e38cd":["3c86b9a5-b882-4e56-8a9d-1817e5fce2cc"],"8f7d9ac7-7f0c-450f-8fc0-3bd5909ec95f":["d3ad3f2e-c44e-4172-96d0-be97661a90ce"],"0e7e1c33-90f3-4a62-85d0-0a8cdbc3efba":["21b65401-1d4e-4fba-a851-91dee4f7f90f"],"170d1322-1ef6-4ac4-9ba1-da2327bf5f0a":["26cc2ed0-fcda-4fb8-a022-02b288d9d196"],"1d889385-3114-4986-bdb7-182f75201f88":["348359fb-d15a-482f-a47d-bbddd0b64fa8","7dee4ffb-8eec-4f9c-9162-ff256330c88c"],"51cb2d9b-670b-41d0-8db4-df7320f31847":["435839f5-2284-485e-9e7b-a03df4d04bed","61755d57-6579-40bb-be8b-1dfaafcdac33"],"a4f9d0d1-60d3-4f5b-afb9-4a7b71110023":["20f3c25d-d34e-456b-821e-ef33399d8674"],"98a38469-9bbd-463f-bf12-2852b385e9dc":["c0dbcefd-0a91-4a7d-af5f-20617c2e56e5"],"e634e2b9-e7a8-4f4e-a97d-77babb7170e7":["899953dd-f64a-4d46-9f83-ed563e045ea8","9fcf5312-01fe-46ea-999a-273b54bc4944"],"b16d3bc0-354c-4448-b790-8ce696e4e046":["e913c667-f400-4063-bb0d-f08c3b56a008","ab4e4132-dcf9-4e35-bb47-3cc9d4e888fe"],"12d2af62-404d-40e2-92e0-1ed9777d7beb":["43a5198e-17f9-426f-831b-fbd162a16f14"],"3a497882-8030-4be2-add2-2a9d98367419":["2e60e207-d662-4982-a252-cb7056c0e388"],"52db83ff-c6d2-431d-a63b-4ff3950a2ebb":["899953dd-f64a-4d46-9f83-ed563e045ea8","9fcf5312-01fe-46ea-999a-273b54bc4944"],"399a978c-17e6-46dd-96bf-04fec645f11a":["e913c667-f400-4063-bb0d-f08c3b56a008","ab4e4132-dcf9-4e35-bb47-3cc9d4e888fe"],"ed0608c5-e649-4320-b3af-a2ce2991cb1c":["551e584b-c6f0-4c17-af3a-6a66e661fe65"],"23babac3-d172-421c-aced-c09775c06dec":["0c108e7e-ec38-4ed0-947f-dcbcce7dc03f","4533e417-6759-4964-bb9f-acbf272386e3"],"a13cd938-9fa3-44ea-9999-0fb3c92a767c":["8c1580d4-275b-4ecc-9747-bd9e9706832c"],"b30e65bc-a83b-4380-8371-49d7b5deeca3":["a57f6cc8-192b-4503-9b63-d509c39ffd0f","4fc5943e-edd8-44ec-8be2-d6bcccca1b40"],"01673f3a-1cef-40b9-8766-c9680c643066":["9d10e8bf-097e-4d8e-829b-b95d0b3c79db"],"1e89ff91-a223-4d8c-b0ba-33a283c56742":["8c5543fb-3461-467a-a1fa-f37eb2c631ea"],"619fe834-e253-4891-8f65-042b3991422a":["74b41b6b-1a65-480f-8302-7e9e71547113"],"341e89c5-a940-4fd0-a8eb-b4aa91033bdf":["b869f06a-bc72-4801-9613-c2f3dd484f8e"],"a0469910-cd3e-4e92-bffb-8ebe4c01555a":["9d10e8bf-097e-4d8e-829b-b95d0b3c79db"],"bed55373-75aa-43e1-9ad3-a831f5c29fde":["74b41b6b-1a65-480f-8302-7e9e71547113"],"c775a019-2ee0-4952-addc-e35f0caffbe8":["b869f06a-bc72-4801-9613-c2f3dd484f8e"],"a02985f6-d60d-4cf7-92fa-618994d73b26":["6c03c816-c17c-4df3-8a2d-d610b409e7dc"],"cea55188-d50a-4fdf-b5a7-459d111e6ce7":["edee0f96-e90a-4512-bdde-6da052e2c31d"],"db7d0d7d-2a54-40be-9c40-ee6e2d4ae512":["862facd2-480b-47c7-b37a-36c9ba209b01"],"fe1423c1-e55e-4efc-8eb1-f97cd09fed23":["389d6b29-c80b-4999-a8f4-009f8829a8ea"],"ac61976d-067c-4d86-abf1-64d9ca756e40":["3578924b-44f6-46b1-ae6b-001e3b00bb57"],"5f2e714b-0ae6-49b9-a739-1768b20bd472":["dae6e6f7-2972-4c25-9254-5fa1f85d3a25"],"a93f9092-6434-487c-9b07-65002e318bc8":["a48ba738-79ba-49fa-bfd9-5668122e51af"],"a49abbde-ae30-4589-bf69-61961f969f64":["20594cc3-c146-4eff-9d11-5036da4bd3e2"],"c02217ae-5d5d-4f41-80ee-33a58f3083e8":["389d6b29-c80b-4999-a8f4-009f8829a8ea"],"2f5283c8-574f-40f9-acec-d64ee9885175":["cada4428-8fa3-4055-924a-f3fe1a7b4a9b"],"cdc663dd-b45e-4a1a-9371-090c2402e9c3":["bad0b181-82d5-49f0-ad3f-ad806ed493d3"],"ad8a336e-470a-4198-a92f-bfcd564f6ea8":["d21a97e9-5cd4-45da-bada-8b928bd4e3a6"],"81481829-ac65-4c93-b7da-26334aa17fe9":["ce78044f-fed7-426a-8f0b-715a4571b529","2add6203-ba18-4186-b38e-2191d1a32cd4"],"f4adc1e1-894c-4a4e-89c9-3f23f8db5da0":["a171231e-8206-423f-a29c-91b0f0b02370","9397b83f-8276-4e91-8b5f-a57c3e6548a9"],"b308c4da-255f-4083-9ffe-35fd4ab599f2":["fa7e9c91-d92b-41b9-8725-9a471c3ba960","5b7e369d-3e95-4d5d-ade4-c9b3c394dd8b"],"52987536-2e82-4480-8862-6f1f0ce71b90":["bb7caef1-dc1d-4010-9443-8222a91dc302","3a7c0f86-76c8-41d3-92fc-63393ed4fd29"],"4c0040d4-64ad-4024-b857-ac9cad7d79eb":["7c553459-9210-42b3-9e8a-161fba23d30b","3faa0e0f-b3c0-468a-9464-fa4d330007e0"],"46453f5d-7b4c-4bb0-8065-b30699802e8e":["052e2ad0-5b6a-4131-8ee3-641ef9df8bc7"],"e9e7dc1f-894c-496a-bd89-7622f46071e9":["31aeeac1-572b-4d8f-a147-117352ff6e83"],"8fcb214c-d28d-4e54-a310-d6ad6e45ccae":["5402f4a4-0741-4593-84a2-da614eae8126"],"4a25949d-4f95-472c-9593-08efd87b57c3":["2b47590b-34b7-4a93-8daa-814c437a6346"],"cbc0999b-bb6c-4ecd-8076-068286e038b3":["fa7e9c91-d92b-41b9-8725-9a471c3ba960","5b7e369d-3e95-4d5d-ade4-c9b3c394dd8b"],"a9ddbff1-29a7-402e-8e6c-bfbc83d79dbb":["eaf34fa8-ddc4-40b1-9235-9b382718842d"],"38a17205-3e5e-422d-84fb-15f382c1e5cd":["5402f4a4-0741-4593-84a2-da614eae8126"],"2ac0bb9b-b93b-4730-9986-4b41b0afa96c":["aae9a847-4e84-4182-bb2d-86e0863d69ce"],"7fa2c2b5-1df0-46b5-bcd3-0e27c9d778fb":["fbb1c8ae-bb98-4a86-9e11-8c9196a4bcf8","94cd33b6-4ea2-4202-a636-d448e25b644b"],"b6e6fbca-6898-4d0d-837d-1149b6917be8":["d94f7171-2144-410d-a128-bcbe8a74faf1"],"d3a8193d-ae81-4b53-a1a7-f7aa0e935284":["dc979649-8464-4bd1-8ec6-9f1c78d6f5c7"],"c8156aeb-42e9-48c6-8919-cddb9086fcdc":["28c1c108-fba1-4696-b6df-394630126cf9"],"b8f2f5c8-7f03-4051-a6a5-8e466e191fd7":["a834cdaf-4a95-40a1-8702-1035fdc9c8a5"],"7bbcee8e-68cb-410a-a3db-83560706e3f5":["4a7c25ef-ba36-4152-8fa8-7d816046deaa"],"131fa4f9-c659-46b6-9a46-c5e09d41471f":["50278fd6-d6ea-4d8f-8680-9783386c487e","e13d383e-9d44-4edd-9889-477e418c6106"],"e3cfab67-00c1-42f1-b612-2bad0d9d9122":["18bb75bb-c909-4315-97b2-79127083778b"],"f48f049f-714f-456b-bc21-1aab5762a224":["c85e3e92-95d1-4f1b-a4f0-51c83bcac9c8"]
    }

In [8]:
def is_seeker(x, kg: KnowledgeGraph):
    return x['Id'] in labels and x['BlockType'] == 'WORD' and kg.nodes[x['ParentId']]['BlockType'] != "CELL" and is_number_like(x['Text'])

def is_truther(x, kg: KnowledgeGraph):
    return x['BlockType'] == "CELL" and "Relationships" in x and kg.nodes[x['Relationships'][0]['Ids'][0]]['BlockType'] == 'WORD' and is_number_like(kg.nodes[x['Relationships'][0]['Ids'][0]]['Text'])

In [9]:
kg = KnowledgeGraph(is_seeker, is_truther, all_blocks)

def seeker_and_truther_have_identical_value(seeker: Seeker, truther: Truther, knowledge_graph):
    return seeker.value == truther.value

def seeker_and_truther_have_similar_values(seeker: Seeker, truther: Truther, knowledge_graph):
    normalized_num1 = normalize_number(seeker.value)
    normalized_num2 = normalize_number(truther.value)
    return normalized_num1 == normalized_num2    

def seeker_and_truther_same_page(seeker: Seeker, truther: Truther, knowledge_graph):
    return seeker.page_num == truther.page_num

#def seekers_in_same_para_truthers_in_same_table_count(count, kg):
#    seeker_set = set(i. for i in kg.seekers)
#    def matching(seeker: Seeker, Truther: Truther):
#        # Get the paragraph that the seeker is in
#        # Get all the seekers that are that paragraph
#        # Get how many in that paragraph are also in the table
#        # If the number of ones that are similar (have another one for exact copy) is equal to count, then the match occurs
#        seeker_para = seeker.closest_layout_block_id
#        potential_seekers = []
#        for rel in kg.nodes[seeker_para]['Relationships']:
#            for child in rel['Ids']:
#                potential_seekers
        

attribute_list = [
    seeker_and_truther_have_identical_value,
    seeker_and_truther_have_similar_values,
    seeker_and_truther_same_page
]

calculated_probabilities = kg.calculate_probabilities(attribute_list, labels)
reward_function = RewardFunction(calculated_probabilities)

In [10]:
h = {}
for x in labels:
    h[kg.nodes[x]['Text']] = [kg.nodes[kg.nodes[y]['Relationships'][0]['Ids'][0]]['Text'] for y in labels[x]]

for i,j in h.items():
    print(i, j)

52% ['52%', '52%']
3% ['3%', '3%']
29% ['29%', '29%']
31% ['31%', '31%', '31%', '31%']
23% ['23%', '23%']
£4.8m ['4,832', '4,832', '4,832']
£5m ['4,991', '4,991', '4,991']
£991k ['991', '991', '991', '991', '991']
£1,103k ['1,103', '1,103', '1,103', '1,103']
£970k ['970', '970', '970', '970', '970']
£991k. ['991', '991', '991', '991', '991']
£26k ['26']
£150k ['(150)', '(150)']
73% ['73%']
18% ['18%']
£529k, ['529']
£546k ['546']
£546k, ['546']
£(84)k ['(84)']
£354k ['354', '354']
£(216)k, ['(216)']
£(81)k, ['(81)', '(81)']
£(27)k, ['(27)', '(27)']
£(70)k. ['(70)', '(70)']
£(42)k ['(42)']
£(112)k ['(112)']
£42k ['(42)']
13% ['13%', '13%', '13%']
16% ['16%', '16%']
3%. ['(3)%']
6% ['6%', '6%']
£54k ['54', '54', '54']
£25k ['25']
£850k ['(850)']
£6k ['6']
£1k ['1']
(£109k ['109']
90% ['90%']
17% ['17%']
22% ['22%']
136 ['136']
154 ['154']
11% ['11%']
£547k ['547']
£916k ['916']
10% ['10%']
7 ['7%', '7%']
9 ['9%']
£527k ['527']
£271k ['271']
£201k ['201', '201']
4% ['4%', '4%']
2% ['2%']


In [11]:
calculated_probabilities

{0: 8.600565179997543e-05,
 4: 0.004260985352862849,
 7: 0.8214285714285714,
 3: 0.2127659574468085}

In [12]:
a = set([i.value for i in kg.seekers])
b = set([i.value for i in kg.truths])
print(b)
print(a.intersection(b))

{'1,944', '1,103', '201', '44%', '834', '187', '527', '498', '787', '22%', '104', '839', '86%', '383', '29,990', '15,349', '154', '131', '51', '178', '38%', '970', '1,479', '434', '916', '272', '118', '61', '457', '1,458', '3%', '52', '163', '164', '9%', '2,776', '215', '5,883', '32 %', '176', '597', '225', '1,075', '774', '1,533', '405', '734', '3', '4,832', '95', '956', '1', '242', '130', '28', '90', '31%', '8', '42%', '670', '586', '1,524', '83%', '116', '2,819', '1,120', '258', '440', '74', '26', '1,597', '211', '0%', '127', '302', '67', '42', '465', '11%', '65', '803', '24', '110', '10', '27%', '954', '20,250', '18%', '280', '931', '662', '54%', '12%', '991', '472', '38', '109', '283', '136', '48', '3,081', '34', '52%', '69', '547', '86', '20%', '28%', '763', '88', '919', '2,264', '424', '524', '82%', '64%', '2,831', '369', '15', '43%', '31', '331', '4%', '108', '277', '200', '20', '1,052', '84', '75', '873', '271', '1,139', '879', '836', '112', '4,413', '887', '2', '695', '659', 

In [13]:
for s in kg.seekers:
    for t in kg.truths:
        if seeker_and_truther_have_similar_values(s, t, kg) and seeker_and_truther_have_identical_value(s, t, kg) and seeker_and_truther_same_page(s, t, kg):
            if t.Id not in labels[s.Id]:
                print(s.value, t.value, s.page_num)

13% 13% 11
13% 13% 11
23% 23% 11
24 24 20
24 24 20
4 4 20
4 4 20
4 4 20
4 4 20
4 4 20


In [14]:
for s in kg.seekers:
    print((s.parent_id, s.grandparent_id, s.closest_layout_block_id))

('82aff880-7c24-4a2f-970a-4f1f6a7e80c7', '49188dd2-4bee-496d-b0d6-52c10d8e7ac7', '49188dd2-4bee-496d-b0d6-52c10d8e7ac7')
('e53e0d37-3cd8-4ab1-9292-2c14d7597d2d', '49188dd2-4bee-496d-b0d6-52c10d8e7ac7', '49188dd2-4bee-496d-b0d6-52c10d8e7ac7')
('c73d4735-8a21-4d51-a135-41ae562321a9', '3dce75f6-47fb-4dd9-88ac-422d10de719c', '3dce75f6-47fb-4dd9-88ac-422d10de719c')
('f1f00189-7b17-4bf5-9c46-0bd6a05f385c', '3dce75f6-47fb-4dd9-88ac-422d10de719c', '3dce75f6-47fb-4dd9-88ac-422d10de719c')
('592adb17-31fb-48c1-b9c0-6f880914e152', 'b320ec76-18d3-44f4-b137-87927660a649', 'b320ec76-18d3-44f4-b137-87927660a649')
('7052cc9e-d433-4371-bb0b-2e8c17b06074', 'bd678541-9abf-47d8-bb0e-18d4d63277f8', 'bd678541-9abf-47d8-bb0e-18d4d63277f8')
('7052cc9e-d433-4371-bb0b-2e8c17b06074', 'bd678541-9abf-47d8-bb0e-18d4d63277f8', 'bd678541-9abf-47d8-bb0e-18d4d63277f8')
('d8ccf665-111b-4418-b9c3-d30f4a1f561e', 'df769efb-5787-425b-a825-960cda04515c', 'df769efb-5787-425b-a825-960cda04515c')
('9303b80f-0682-455f-b36a-44a971

In [15]:
for s in kg.seekers:
    print(s.page_num)

7
7
7
7
7
8
8
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
13
13
15
15
15
18
18
18
18
18
18
18
18
19
20
20
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
23
23
23
23
24
24
28
30
30
32
32
40
40
42


In [16]:
for i in all_blocks:
    print(i['Page'])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [19]:
for i in kg.seekers:
    for j in kg.truths:
        if not seeker_and_truther_have_similar_values(i, j, kg) and  not seeker_and_truther_have_identical_value(i, j, kg) and  seeker_and_truther_same_page(i, j, kg) and j.Id in labels[i.Id]:
            print(i.value, j.value, i.page_num)

£54k 54 11
£26k 26 12
£25k 25 12
£6k 6 12
£1k 1 12
£9k 9 18
£37k 37 20
£75k 75 20
£698k 698 21
£5.8m 5,788 21
£479k 479 21
£439k 439 24
£457k 457 24
£51k 51 30
£49k 49 30
£313k 313 32
